In [ ]:
# script to make xlsx sheets for input of exam marks
#
# place this ipynb in the same dir as:
# (1) a list of student reg nos, with filename <courseCode>.unit_student_profile, where <courseCode> is e.g. 223
# (2) the _markslist.json file produced by exam24.cls, which must also contain <courseCode> somewhere within its filename
# run the code below to generate an xlsx ready for input of part-marks

In [43]:
import glob
import os
import json
import pandas
import numpy as np
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
from openpyxl.styles.borders import Border, Side

In [28]:
def NtoC(n):
    return chr(ord('@')+n)

In [131]:
def CreateMarkSheet(courseCode, studentsFilename, marksFilename):
    students = pandas.read_csv(studentsFilename, sep='\t+', engine='python')
    partQs = pandas.read_json(marksFilename)
    partQcounts = partQs.groupby('question').size()
    
    wb = Workbook()
    ws = wb.active
    
    # openpyxl defines cell "A1" to be coordinate (1,1) 
    sColStart = 1 # student IDs 
    sColEnd = len(students.columns)
    pColStart = sColEnd + 2 # part Qs
    pColEnd = pColStart + len(partQs)
    qColStart = pColEnd + 1 # Qs
    qColEnd = qColStart + len(partQcounts)
    sumCol = qColEnd + 1
    percCol = sumCol + 1
    
    headRow = 1
    maxRow = 2
    avgRow = maxRow + len(students) + 2
    
    inputSide = Side(style='thin', color="95B3D7")
    inputBorder = Border(left=inputSide, right=inputSide, top=inputSide, bottom=inputSide)
    
    # write in students
    for i,(colName,col) in enumerate(students.items()):
        ws.cell(maxRow,i+1).value = colName
        ws.cell(maxRow,i+1).font = Font(bold=True)
        ws.column_dimensions[NtoC(i+1)].width = 15
        for j,v in enumerate(col):
            ws.cell(maxRow+j+1,i+1).value = str(v)
    
    # average row 
    ws.cell(avgRow, pColStart-1).value="avg %"
    ws.cell(avgRow, pColStart-1).font = Font(bold=True)
    
    # write in part-q headings, maxes, input cells and averages
    for i,row in partQs.iterrows():
        _i = pColStart+i
        label = str(row.question) if row.part==0 else str(row.question)+NtoC(row.part).lower()
        ws.cell(headRow,_i).value = label
        ws.cell(headRow,_i).font = Font(bold=True)
        ws.cell(headRow,_i).alignment = Alignment(horizontal="right")
        ws.cell(maxRow,_i).value = row.marks
        ws.column_dimensions[NtoC(_i)].width = 5
        for j in range(len(students)):
            ws.cell(maxRow+j+1,_i).border = inputBorder
            ws.cell(maxRow+j+1,_i).font = Font(color="366092")
        ws.cell(avgRow,_i).value = "=AVERAGE(" + NtoC(_i) + str(maxRow+1) + ":" + NtoC(_i) + str(maxRow+len(students)) + ")/" + NtoC(_i) + str(maxRow) + "*100"
    
    # write in the q headings, maxes and averages
    cumsumPartCount = partQcounts.cumsum()
    cumsumPartCount = pandas.concat([pandas.Series([0]), cumsumPartCount])
    for q,partCount in partQcounts.items():
        _i = qColStart+q-1
        pFirst = pColStart + cumsumPartCount[q-1]
        pLast = pColStart + cumsumPartCount[q] - 1
        ws.cell(headRow,_i).value = str(q)
        ws.cell(headRow,_i).font = Font(bold=True)
        ws.cell(headRow,_i).alignment = Alignment(horizontal="right")
        ws.column_dimensions[NtoC(_i)].width = 5
        for j in range(maxRow,maxRow+len(students)+1):
            ws.cell(j,_i).value = "=IF(" + NtoC(pFirst) + str(j) + '="","",SUM(' + NtoC(pFirst) + str(j) + ":" + NtoC(pLast) + str(j) + '))'
        ws.cell(avgRow,_i).value = "=AVERAGE(" + NtoC(_i) + str(maxRow+1) + ":" + NtoC(_i) + str(maxRow+len(students)) + ")/" + NtoC(_i) + str(maxRow) + "*100"
    
    # write in the sum and perc columns
    ws.cell(headRow, sumCol).value = "sum"
    ws.cell(headRow, sumCol).font = Font(bold=True)
    ws.cell(headRow, percCol).value = "%"
    ws.cell(headRow, percCol).font = Font(bold=True)
    ws.column_dimensions[NtoC(sumCol)].width = 5
    ws.column_dimensions[NtoC(percCol)].width = 5
    for j in range(maxRow,maxRow+len(students)+1):
        ws.cell(j,sumCol).value = "=IF(" + NtoC(pColStart) + str(j) + '="","",SUM(' + NtoC(pColStart) + str(j) + ":" + NtoC(pColEnd-1) + str(j) + "))" 
    for j in range(maxRow+1,maxRow+len(students)+1):
        ws.cell(j,percCol).value = "=IF(" + NtoC(pColStart) + str(j) + '="","",' + NtoC(sumCol) + str(j) + "/" + NtoC(sumCol) + str(maxRow) +"*100)"
    for _i in (sumCol, percCol):
        ws.cell(avgRow,_i).value = "=AVERAGE(" + NtoC(_i) + str(maxRow+1) + ":" + NtoC(_i) + str(maxRow+len(students)) + ")/" + NtoC(_i) + str(maxRow) + "*100"
        ws.cell(avgRow,_i).value = "=AVERAGE(" + NtoC(_i) + str(maxRow+1) + ":" + NtoC(_i) + str(maxRow+len(students)) + ")" 
    
    wb.save(courseCode + "_marks.xlsx")

In [132]:
# for each X.unit_student_profile, find a Y_marklist.json file containing X as a substring of Y
# pair those two and generate a marksheet
studentFilenames = glob.glob(os.path.join(".","*.unit_student_profile"))
marksFilenames = glob.glob(os.path.join(".","*_marklist.json"))
for studentsFilename in studentFilenames:
    courseCode = os.path.split(studentsFilename)[1].split(".")[0]
    marksFilename = ""
    for _marksFilename in marksFilenames:
        if courseCode in _marksFilename:
            marksFilename = _marksFilename
            break
    if marksFilename:
        CreateMarkSheet(courseCode, studentsFilename, marksFilename)
    else:
        print("No marks file found for %s" % studentsFilename, "with course code %s" % courseCode)
    